In [2]:
from __future__ import absolute_import
import re
import kenlm
from heapq import heapify
from six.moves import range

In [3]:
BEAM_WIDTH = 1024
MODEL = None

In [4]:
def get_model():
    global MODEL
    if MODEL is None:
        MODEL = kenlm.Model('lm.binary')
    return MODEL


In [5]:
def words(text):
    "List of words in text."
    return re.findall(r'\w+', text.lower())


In [6]:
def log_probability(sentence):
    "Log base 10 probability of `sentence`, a list of words"
    return get_model().score(' '.join(sentence), bos = False, eos = False)

In [7]:
def correction(sentence):
    "Most probable spelling correction for sentence."
    layer = [(0,[])]
    for word in words(sentence):
        layer = [(-log_probability(node + [cword]), node + [cword]) for cword in candidate_words(word) for priority, node in layer]
        heapify(layer)
        layer = layer[:BEAM_WIDTH]
    return ' '.join(layer[0][1])

In [8]:
def candidate_words(word):
    "Generate possible spelling corrections for word."
    return (known_words([word]) or known_words(edits1(word)) or known_words(edits2(word)) or [word])


In [9]:
def known_words(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

In [10]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [11]:
def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [12]:
with open('words.txt') as f:
    WORDS = set(words(f.read()))

In [ ]:
text="o have you ever gazed at the mood and stars and wondered wonderd what it would be like to floawt in the mighcro gravity of space or walk on the moon for thousands of years humans could only imagine whath those experiences would be like but in the twentieth century the dreams of human space flight became a reality when in nineteen sixty one the soviat unions eurigagarin first orbited the earth we an eight years of his launch neel arm strong was to utter what may be the most significant line in human history that's one small step for man one giant leap from mankind as he stepped on to the moon surface ol i'm churlain a professor in the department of healthinyr performance that the universty of oston and i will be your guide as we explored the history of human spatefloght from the musings of the authors an antiquity about space to the development of the powerful rockets which carried humans inti space through the earli successes and failures of the american and soviat union space programs to the permanent human presence of space in a variety of spacestations this course wo faseny an entertaying wil perbyinou insighte and little known facts about human spacefly ou te ar comination especially de bilop vilios and a varity of reedings dhi carswoln hanshere knowledce about the history of emen spaceflke i invite you to watch the demestration vide to learn about the basic design in the course and to see what i dis course is a must regardless of your background for anyone in stoid in humin spacellem"
correction(text)